# AnchorViewer

In [1]:
# imports
import pathlib

import numpy as np
from torchvision.models.detection.transform import GeneralizedRCNNTransform

from pytorch_faster_rcnn_module.datasets import ObjectDetectionDataSet
from pytorch_faster_rcnn_module.transformations import Clip, ComposeDouble
from pytorch_faster_rcnn_module.transformations import FunctionWrapperDouble
from pytorch_faster_rcnn_module.transformations import normalize_01
from pytorch_faster_rcnn_module.utils import get_filenames_of_path
from pytorch_faster_rcnn_module.visual import AnchorViewer

In [2]:
# root directory
root = pathlib.Path('pytorch_faster_rcnn_module/data/heads')

In [3]:
# input and target files
inputs = get_filenames_of_path(root / 'input')
targets = get_filenames_of_path(root / 'target')

inputs.sort()
targets.sort()

In [4]:
# mapping
mapping = {
    'head': 1,
}

In [5]:
# transforms
transforms = ComposeDouble([
    Clip(),
    # AlbumentationWrapper(albumentation=A.HorizontalFlip(p=0.5)),
    # AlbumentationWrapper(albumentation=A.RandomScale(p=0.5, scale_limit=0.5)),
    # AlbumentationWrapper(albumentation=A.VerticalFlip(p=0.5)),
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

In [6]:
# dataset building
dataset = ObjectDetectionDataSet(inputs=inputs,
                                 targets=targets,
                                 transform=transforms,
                                 use_cache=False,
                                 convert_to_format=None,
                                 mapping=mapping)

In [7]:
# transforms
transform = GeneralizedRCNNTransform(min_size=1024,
                                     max_size=1024,
                                     image_mean=[0.485, 0.456, 0.406],
                                     image_std=[0.229, 0.224, 0.225])

In [8]:
image = dataset[0]['x']  # ObjectDetectionDataSet
feature_map_size = (512, 32, 32)
anchorviewer = AnchorViewer(image=image,
                            rcnn_transform=transform,
                            feature_map_size=feature_map_size,
                            anchor_size=((128, 256, 512),),
                            aspect_ratios=((0.5, 1.0, 2.0),)
                            )
anchorviewer.napari()